In [3]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [4]:
df_jan = pd.read_parquet('./yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('./yellow_tripdata_2023-02.parquet')

In [5]:
len(df_jan)

3066766

#### 1- Read the data for January. How many columns are there?

In [6]:
len(df_jan.columns)

19

#### 2- Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [7]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [8]:
df_jan["duration"] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan["duration"] = df_jan.duration.dt.total_seconds() / 60

In [9]:
df_jan.duration.std()

42.59435124195458

#### 4- Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [10]:
100*len(df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)])/len(df_jan)

98.1220282212598

In [11]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)].copy()

#### 4- One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [12]:
categorical = ['PULocationID', 'DOLocationID']

df_jan[categorical] = df_jan[categorical].fillna(-1).astype('int')

In [13]:
df_jan[categorical] = df_jan[categorical].astype('str')

In [14]:
train_dicts = df_jan[categorical].to_dict(orient='records')

In [15]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts) 

In [16]:
X_train.shape

(3009173, 515)

#### 5- Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [17]:
y_train = df_jan.duration.values

In [18]:
len(dv.feature_names_)

515

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [20]:
y_pred = lr.predict(X_train)

In [24]:
mean_squared_error(y_train, y_pred, squared=False)

7.649261937621321

#### 6- Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [31]:
categorical = ['PULocationID', 'DOLocationID']

def prepare_data(df):
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [32]:
df_prep = prepare_data(df_feb)

In [34]:
val_dicts = df_prep[categorical].to_dict(orient='records')

In [35]:
X_val = dv.transform(val_dicts) 

In [36]:
y_pred = lr.predict(X_val)

In [37]:
y_val = df_prep.duration.values

In [38]:
mean_squared_error(y_val, y_pred, squared=False)

7.811817646307258